In [1]:
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import pandas as pd
from pprint import pprint

In [127]:
options = webdriver.FirefoxOptions()

driver = webdriver.Firefox(options=options)
# driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
#     "source": """
#             Object.defineProperty(navigator, 'webdriver', {
#               get: () => undefined
#             })
#             """
# })
driver.get("https://learn.microsoft.com/en-us/azure/virtual-machines/")

In [128]:
# get list of urls for crawling
instances = driver.find_element(By.ID, "title-8-1_1-2")
tree_group = instances.find_element(By.CLASS_NAME, "tree-group")
items = tree_group.find_elements(By.TAG_NAME, 'li')
urls = []
for item in items:
    if item.text == "Generation 2 VMs":
        break
    if item.get_attribute("role") == "none":
        urls.append(item.find_element(By.TAG_NAME, 'a').get_attribute("href"))
        # print(item.find_element(By.TAG_NAME, 'a').get_attribute("href"))

In [130]:
# num = 44
import time
fileName = []
series = []

for url in urls:
    # print(num, len(urls), url)
    num = num + 1
    driver.get(url)
    time.sleep(5)
    tables = driver.find_elements(By.TAG_NAME, 'table')
    sery = driver.find_element(By.TAG_NAME,'h1').text
    if sery == 'Previous generations of virtual machine sizes':
        sery = 'prev gen'
    series.append(sery)
    for table in tables:

        # find table headers
        ths = table.find_elements(By.TAG_NAME, 'th')
        if ths[0].get_attribute("innerText") !="Size":
            continue
        content = ""
        tmp = []
        for th in ths:
            tmp.append(th.get_attribute("innerText"))
        content += ",".join(tmp) + "\n"

        trs = table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
        # n = 0
        for tr in trs:
            tds = tr.find_elements(By.TAG_NAME, 'td')
            # if n == 0:
            # title = tds[0].get_attribute("innerText").replace("\n", "").replace(" ", "").replace("*","")
            fileName.append(title)
            tmp = []
            for td in tds:
                t = td.get_attribute("innerText")
                sup = td.find_elements(By.TAG_NAME, 'sup')
                if len(sup) > 0:
                    if "," in t:
                        t = t[:-4]
                        title = t
                    else:
                        t = t[:-1]
                tmp.append(t)
            content += ",".join(tmp) + "\n"
        f = open(f'./data/azure_raw/{sery}_{title}.csv', 'w', encoding="utf-8")
        f.write(content)
        f.close()

# Get Azure vCPU/core rate data

In [135]:
# get vCPU/core rate data
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)
driver.get("https://learn.microsoft.com/en-us/azure/virtual-machines/acu")

In [ ]:
# table of rates
import numpy as np
table = driver.find_element(By.XPATH, "//table[@aria-label='Table 1']")
headers = table.find_elements(By.TAG_NAME, 'th')
headers = [i.text for i in headers]
rate_tds = table.find_elements(By.TAG_NAME, 'tbody')
for td in rate_tds:
    t = td.get_attribute("innerText")
    t = t.replace("*","")
    data = t.split("\n") # split into separate rows
data2 = []
for i in data:
    tmp = i.split("\t")
    for d in tmp:
        if ":" in d:
            d = d.split(":")
            d[0] = float(d[0])
            d[1] = float(d[1])
            d = d[0] / d[1]
            tmp[2] = d

    data2.append(tmp)
df = pd.DataFrame(columns=headers,data=data2)
df['Retirement Date'] = "NaN"

# retireing vm machine table

table = driver.find_element(By.XPATH, "//table[@aria-label='VM Series Retiring']")
headers = table.find_elements(By.TAG_NAME, 'th')
headers = [i.text for i in headers]
rate_tds = table.find_elements(By.TAG_NAME, 'tbody')
for td in rate_tds:
    t = td.get_attribute("innerText")   
    t = t.replace("*","") # t 本來就是全部的資料了
    data = t.split("\n") # split into separate rows
data2 = []
for i in data:
    tmp = i.split("\t")
    for d in tmp:
        if ":" in d:
            d = d.split(":")
            d[0] = float(d[0])
            d[1] = float(d[1])
            d = d[0] / d[1]
            tmp[2] = d

    data2.append(tmp)
df2 = pd.DataFrame(columns=headers,data=data2)

# merge two dataframes and convert datetime formats
df = pd.concat([df,df2],ignore_index=True)
df['Retirement Date'] = pd.to_datetime(df['Retirement Date'],yearfirst=True)
df.to_csv('./data/azure/azure_acu.csv')


# All-data processing

In [20]:
rateTb = pd.read_csv('./data/azure/azure_acu.csv')
sku = rateTb['SKU Family']
sku = [i.replace("_","") for i in sku]
rateTb['SKU Family'] = sku



0     1.0
1     1.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     2.0
8     2.0
9     2.0
10    2.0
11    2.0
12    2.0
13    2.0
14    2.0
15    2.0
16    2.0
17    2.0
18    2.0
19    2.0
20    2.0
21    2.0
22    2.0
23    2.0
24    1.0
25    1.0
26    2.0
27    1.0
28    1.0
29    1.0
30    1.0
31    1.0
32    1.0
33    2.0
34    2.0
35    2.0
36    2.0
37    1.0
38    1.0
39    1.0
40    1.0
41    1.0
42    1.0
Name: vCPU: Core, dtype: float64

In [10]:
files = [i for i in os.listdir('./data/azure_raw') if '.csv' in i]
files.sort()
files
# result = pd.read_csv('./data/azure_raw/'+ files[0])
# for file in files:
#     df = pd.read_csv('./data/azure_raw/'+file)

#[['Size','vCPU','Memory: GiB','Temp storage (SSD) GiB','GPU memory: GiB','GPU']]
#[['Size', 'Physical CPU cores', 'Processor', 'Memory (GB)','Base CPU frequency (GHz)', 'Temp storage (TB)']]

['Av2-series_Standard_A8m_v2.csv',
 'Av2-series_Standard_ND24rs.csv',
 'B-series burstable virtual machine sizes_Standard_B20ms.csv',
 'B-series burstable virtual machine sizes_Standard_ND24rs.csv',
 'DCasv5 and DCadsv5-series confidential VMs_Standard_DC96ads_v5.csv',
 'DCasv5 and DCadsv5-series confidential VMs_Standard_DC96as_v5.csv',
 'DCasv5 and DCadsv5-series confidential VMs_Standard_ND24rs.csv',
 'DCsv2-series_Standard_DC8_v2.csv',
 'DCsv2-series_Standard_ND24rs.csv',
 'DCsv3 and DCdsv3-series_Standard_DC48ds_v3.csv',
 'DCsv3 and DCdsv3-series_Standard_DC48s_v3.csv',
 'DCsv3 and DCdsv3-series_Standard_ND24rs.csv',
 'Dasv5 and Dadsv5-series_Standard_D96ads_v5.csv',
 'Dasv5 and Dadsv5-series_Standard_D96as_v5.csv',
 'Dasv5 and Dadsv5-series_Standard_ND24rs.csv',
 'Dav4 and Dasv4-series_Standard_D96a_v4.csv',
 'Dav4 and Dasv4-series_Standard_D96as_v4.csv',
 'Dav4 and Dasv4-series_Standard_ND24rs.csv',
 'Ddv4 and Ddsv4-series_Standard_D64d_v4.csv',
 'Ddv4 and Ddsv4-series_Standard_